In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import networkx as nx
%matplotlib inline
# from DYNASMART_Process import *
from Modules import DYNASMART_Process as dy
# import preprocessing_read_skim_file as rs
from Modules import AV_functions as av
from Modules import Solve_DARP as sod
from Modules import Postprocess_DARP as pod
from Modules import Preprocess_DARP as prd
from Modules import DARP_to_DYNA as dtd
from gurobipy import *
from Modules import *
import datetime
import requests
import math
import importlib
import random
%load_ext autoreload
%autoreload 2

In [2]:
#Read in the basic traveler and trips informationa and joint the houshold id
# traveler_info=pd.read_csv('input/traveler.csv')
personData=pd.read_csv('input/personData_1.csv')
person_type_dic=dict(zip(personData.person_id,personData.type))
traveler_trips=pd.read_csv('input/traveler_trip_info_improved.csv',index_col=0)
num_zones,num_nodes,num_links,node_detail,node_id,link_detail,link_id=dy.read_network('input/network.dat')
nodexy=dy.read_xy('input/xy.dat')
superzone_filepath='Input/SuperZone.dat'
linktraveltime_filepath='Input/output_td_linktraveltime.dat'
superzone_map=dy.read_superzone_info(superzone_filepath)
origin_path='Input/origin.dat'
destination_path='Input/destination.dat'
origins,destinations,origin_links=dy.read_origin_destion(origin_path,destination_path)
origin_links['length']=origin_links.apply(lambda row: 
                                          link_detail[int(link_id[node_id[row.u_node],node_id[row.d_node]])][0,4],axis=1)
VPNRSkim_folder_filepath='Input/vehicularPNRSkim'
Vehicular_Skim,Vehicular_Skim_Dict=dy.read_vehicle_skim(VPNRSkim_folder_filepath)
transit_skim_folderpath='Input/'
TransitMazTazFlag,drivingweight,walkingweight,\
TransitSkimTimeIntervalLength,MaxNumTransitSkimTimeInterval,WalkSpeed,transit_zone_candidates,\
Transit_AB_Cost_Skim,Transit_AB_Time_Skim,three_link_walk=dy.read_transitskim(transit_skim_folderpath)
Transit_AB_Cost_Skim_Dict=dy.convert_skim_dataframe_to_dict(Transit_AB_Cost_Skim,TransitSkimTimeIntervalLength)
Transit_AB_Time_Skim_Dict=dy.convert_skim_dataframe_to_dict(Transit_AB_Time_Skim,TransitSkimTimeIntervalLength)
transit_zone_dict={}
for link_zone in three_link_walk.three_link_zone.unique():
    transit_zone_dict[link_zone]=list(three_link_walk[three_link_walk.three_link_zone==link_zone]['transit_zone'].astype(int))
three_link_walk_dict={}
for index,group in three_link_walk.groupby(['three_link_zone']):
    three_link_walk_dict[index]={}
    for inx,row in group.iterrows():
        three_link_walk_dict[index][row.transit_zone]=row.distance
intrasuperzone_vehicle_filepath='Input/intrasuperzone_vehicle.dat'
intrasuperzone_path_filepath='Input/intrasuperzone_path.dat'
drivingcost_per_mile=0.5
intrasuperzone_info,intrasuperzone_path_dic=dtd.read_intrasuperzone_files(intrasuperzone_vehicle_filepath,intrasuperzone_path_filepath)
traveler_trips=prd.compare_mode_utlity(traveler_trips,TransitMazTazFlag,
                        three_link_walk_dict,Transit_AB_Cost_Skim_Dict,
                        Transit_AB_Time_Skim_Dict,TransitSkimTimeIntervalLength,
                        Vehicular_Skim_Dict,superzone_map,drivingcost_per_mile,transit_zone_dict)

56 57 72


In [14]:
car_trips=traveler_trips.loc[(traveler_trips.tripmode<=6) &(traveler_trips.driver_passenger_flag>=1)]
car_trips=car_trips.loc[(car_trips.orig_taz !=car_trips.dest_taz) | (car_trips.trip_counter==1)]
car_transit_trips=car_trips.append(traveler_trips[traveler_trips.tripmode>8])
# car_transit_trips=prd.compare_mode_utlity(car_transit_trips,TransitMazTazFlag,
#                         three_link_walk_dict,Transit_AB_Cost_Skim_Dict,
#                         Transit_AB_Time_Skim_Dict,TransitSkimTimeIntervalLength,
#                         Vehicular_Skim_Dict,superzone_map,drivingcost_per_mile,transit_zone_dict)
random.seed(100)
car_transit_trips_sample=car_transit_trips[car_transit_trips.hh_id.isin(random.sample(list(car_transit_trips.hh_id.unique()),3000))]
car_trips_sample=car_trips[car_trips.hh_id.isin(random.sample(list(car_transit_trips.hh_id.unique()),3000))]

In [54]:
car_transit_trips_sample.groupby('actual_mode').count()

,person_id,num_trips,value_of_time,trip_counter,ActivityTime,tripmode,orig_purpose,dest_purpose,orig_maz,orig_taz,...,Diff_bet_starttime,travel_time,person_type,earlyarr_penalty,latearr_penalty,early_penalty_threshold,late_penalty_threshold,transit_utility,car_utility,predicted_mode
actual_mode,,,,,,,,,,,,,,,,,,,,,
Car,17869,17869,17869,17869,17869,17869,17869,17869,17869,17869,...,17869,17869,17869,17869,17869,17869,17869,17869,17869,17869
NonCar,1187,1187,1187,1187,1187,1187,1187,1187,1187,1187,...,1187,1187,1187,1187,1187,1187,1187,1187,1187,1187


In [ ]:
for index, group in car_transit_trips.groupby('hh_id'):
    if abs(len(group[group.actual_mode=='Car'])-len(group[group.predicted_mode=='Car']))<10 and len(group)>25:
        print(index,len(group),abs(len(group[group.actual_mode=='Car'])-len(group[group.predicted_mode=='Car'])))

In [49]:
output_flag=0
# target_hh_id=2744264
min_length=15
max_length=20
single_model_runtime=60*3
drivingcost_per_mile=0.5
reward_mode=2
run_mode=0
darp_solutions=[]
cav_use_mode=0
num_time_interval=1
TL=[i*1440/num_time_interval for i in range(num_time_interval)]
TU=[(i+1)*1440/num_time_interval for i in range(num_time_interval)]
num_cav=1
share_ride_factor=1.5
time_window_flag=0
# target_trips=traveler_trips.loc[(traveler_trips.tripmode<=6) &(traveler_trips.driver_passenger_flag>0)]
# target_trips=target_trips[(target_trips.orig_taz!=target_trips.dest_taz) | (target_trips.trip_counter==1)]
# target_trips=target_trips.append(traveler_trips[traveler_trips.tripmode>8])
# target_trips.value_of_time=target_trips.value_of_time*0.8
# car_transit_trips_sample.value_of_time=car_transit_trips_sample.value_of_time*0.7
for share_ride_factor in [1,1.25,1.5]:
    print('********************************')
    print(share_ride_factor)
    route_info,darp_solutions=\
    sod.get_route_info_allhh(car_trips_sample,
                             output_flag,min_length,max_length,single_model_runtime,drivingcost_per_mile,
                             reward_mode,run_mode,cav_use_mode,num_time_interval,num_cav,share_ride_factor
                             ,time_window_flag,Vehicular_Skim_Dict,
                             Transit_AB_Cost_Skim_Dict,Transit_AB_Time_Skim_Dict,three_link_walk_dict,
                             superzone_map,TL,TU,
                             transit_zone_dict,transit_zone_candidates,TransitMazTazFlag,
                             TransitSkimTimeIntervalLength )

    pod.analysis_network_level_results(route_info,darp_solutions,car_trips_sample,
                                       Vehicular_Skim_Dict,superzone_map,drivingcost_per_mile)

********************************
1
Estimate Route for the  0 th household  2018-07-17 19:47:11.267191
Estimate Route for the  100 th household  2018-07-17 19:47:44.857613
Estimate Route for the  200 th household  2018-07-17 19:48:16.442831
Estimate Route for the  300 th household  2018-07-17 19:48:43.405578
Estimate Route for the  400 th household  2018-07-17 19:49:11.361257
Estimate Route for the  500 th household  2018-07-17 19:49:33.479510
Estimate Route for the  600 th household  2018-07-17 19:49:56.472819
Estimate Route for the  700 th household  2018-07-17 19:50:21.207222
Estimate Route for the  800 th household  2018-07-17 19:50:44.445544
Estimate Route for the  900 th household  2018-07-17 19:51:05.834766
Estimate Route for the  1000 th household  2018-07-17 19:51:27.047965
Estimate Route for the  1100 th household  2018-07-17 19:51:46.743085
Estimate Route for the  1200 th household  2018-07-17 19:52:10.358428
Estimate Route for the  1300 th household  2018-07-17 19:52:29.0974

In [50]:
output_flag=0
# target_hh_id=2744264
min_length=15
max_length=20
single_model_runtime=60*3
drivingcost_per_mile=0.5
reward_mode=1
run_mode=0
darp_solutions=[]
cav_use_mode=0
num_time_interval=1
TL=[i*1440/num_time_interval for i in range(num_time_interval)]
TU=[(i+1)*1440/num_time_interval for i in range(num_time_interval)]
num_cav=1
share_ride_factor=1.5
time_window_flag=0
# target_trips=traveler_trips.loc[(traveler_trips.tripmode<=6) &(traveler_trips.driver_passenger_flag>0)]
# target_trips=target_trips[(target_trips.orig_taz!=target_trips.dest_taz) | (target_trips.trip_counter==1)]
# target_trips=target_trips.append(traveler_trips[traveler_trips.tripmode>8])
# target_trips.value_of_time=target_trips.value_of_time*0.8
# car_transit_trips_sample.value_of_time=car_transit_trips_sample.value_of_time*0.7
# for share_ride_factor in [1,1.25,1.5]:
print('********************************')
print(share_ride_factor)
route_infomodechoice,darp_solutionsmodechoice=\
sod.get_route_info_allhh(car_transit_trips_sample,
                         output_flag,min_length,max_length,single_model_runtime,drivingcost_per_mile,
                         reward_mode,run_mode,cav_use_mode,num_time_interval,num_cav,share_ride_factor
                         ,time_window_flag,Vehicular_Skim_Dict,
                         Transit_AB_Cost_Skim_Dict,Transit_AB_Time_Skim_Dict,three_link_walk_dict,
                         superzone_map,TL,TU,
                         transit_zone_dict,transit_zone_candidates,TransitMazTazFlag,
                         TransitSkimTimeIntervalLength )



********************************
1.5
Estimate Route for the  0 th household  2018-07-17 20:35:09.885699
Estimate Route for the  100 th household  2018-07-17 20:36:14.097354
Estimate Route for the  200 th household  2018-07-17 20:37:19.342058
Estimate Route for the  300 th household  2018-07-17 20:38:25.071795
Estimate Route for the  400 th household  2018-07-17 20:39:28.883425
Estimate Route for the  500 th household  2018-07-17 20:40:19.255286
Estimate Route for the  600 th household  2018-07-17 20:41:13.022447
Estimate Route for the  700 th household  2018-07-17 20:41:55.376132
Estimate Route for the  800 th household  2018-07-17 20:42:44.050220
Estimate Route for the  900 th household  2018-07-17 20:43:26.872935
Estimate Route for the  1000 th household  2018-07-17 20:44:05.224370
Estimate Route for the  1100 th household  2018-07-17 20:44:42.103702
Estimate Route for the  1200 th household  2018-07-17 20:45:29.148685
Estimate Route for the  1300 th household  2018-07-17 20:46:07.97

In [55]:
pod.analysis_network_level_results(route_infomodechoice,darp_solutionsmodechoice,car_trips_sample,
                                   Vehicular_Skim_Dict,superzone_map,drivingcost_per_mile)

Number of AV Trips 22404
Number of Shared Rides 310.0
Total VMT 162222.538000003
Number of occupied trips   18081
Number of unoccupied trips  4323
Number of transit trips -38.0
total_convention_vehicle_driving_time 1310.4933333333283
total_convention_vehicle_driving_distance 133666.38099999996
total_AV_driving_time 1559.418333333341
total_AV_unoccupied_driving_time 286.98333333333375
total_AV_driving_distance 162270.12600000302
total_AV_unoccupied_driving_distance 30273.89699999995


In [43]:
route_info.columns

Index(['orig_zone', 'dest_zone', 'orig_node_index', 'dest_node_index',
       'person_id', 'origin_arrival_time', 'dest_arrival_time',
       'dest_expected_arrival_time', 'value_of_time', 'start_time',
       'Activity_Time', 'hh_id', 'hh_vehicle_id', 'shared_ride_flag',
       'pickup_trip_flag', 'transit_utility', 'car_utility',
       'intrasuperzone_flag', 'seg_index', 'veh_seg_index'],
      dtype='object')

Number of Trips 21827
Number of Shared Rides 327
Total VMT 160581.2780000041
Number of occupied trips   18057
Number of unoccupied trips  3770
Number of transit trips 0
total_convention_vehicle_driving_time 1310.4933333333283
total_convention_vehicle_driving_distance 133666.38099999996
total_AV_driving_time 1554.0033333333445
total_AV_unoccupied_driving_time 255.59166666666633
total_AV_driving_distance 160599.42900000414
total_AV_unoccupied_driving_distance 26749.142999999953


In [28]:
route_info.pickup_trip_flag.sum()

17727

In [29]:
route_info.shared_ride_flag.sum()

327

In [ ]:
route_info.

In [ ]:
len(route_info_modechoice)

In [ ]:
len(car_transit_trips_sample)

In [ ]:
len(route_info_modechoice)

In [ ]:
total_num_pickup_trip=0
for keys,value in darp_solutions_modechoice.items():
#     print(value)
    total_num_pickup_trip+=value['num_pickup_trips']


In [ ]:
total_num_pickup_trip

In [ ]:
target_trips.groupby('predicted_mode').count()

In [ ]:
route_info_modechoice08.to_csv('route_info_modechoice08')


In [ ]:
pod.save_obj(darp_solutions_modechoice08,'darp_solutions_modechoice08','Output/')

In [ ]:
pickle.dump(darp_solutions_modechoice08,'darp_solutions_modechoice09',)

In [ ]:
len(route_info_modechoice08[route_info_modechoice08.person_id==0])

In [ ]:
drivingcost_per_mile=0.5

In [ ]:
modechoice_target_trips=traveler_trips.loc[(traveler_trips.tripmode<=6) &(traveler_trips.driver_passenger_flag>0)]
modechoice_target_trips=modechoice_target_trips[(modechoice_target_trips.orig_taz!=modechoice_target_trips.dest_taz) | (modechoice_target_trips.trip_counter==1)]
modechoice_target_trips=modechoice_target_trips.append(traveler_trips[traveler_trips.tripmode>8])

In [ ]:
modechoice_target_trips=prd.compare_mode_utlity(modechoice_target_trips,TransitMazTazFlag,
                                               three_link_walk_dict,Transit_AB_Cost_Skim_Dict,
                                               Transit_AB_Time_Skim_Dict,TransitSkimTimeIntervalLength,
                                               Vehicular_Skim_Dict1,superzone_map,drivingcost_per_mile,transit_zone_dict)

In [ ]:
modechoice_target_trips.value_of_time=modechoice_target_trips.value_of_time*0.8

In [ ]:
modechoice_target_trips['car_utility']=modechoice_target_trips.apply(lambda row: 
        (prd.estimate_single_car_trip_cost(row.orig_taz,row.dest_taz,row.starttime,row.value_of_time,
        Vehicular_Skim_Dict,0,superzone_map,drivingcost_per_mile)), axis=1)
    
modechoice_target_trips['predicted_mode']=modechoice_target_trips.apply(lambda row:
    'NonCar' if row.transit_utility<row.car_utility else 'Car',axis=1)

In [ ]:
modechoice_target_trips.groupby(['predicted_mode']).count()

In [ ]:
run_name='allhhcartrip1.5/'
output_filepath='Output/'+run_name
route_info_allcar=pd.read_csv(output_filepath+'route_info.csv')
darp_solution=pod.load_obj('darp_solutions',output_filepath)
average_value_of_time=round(traveler_trips.value_of_time.mean(),4)
external_factor='50'
dtd.write_darp_solution_to_file(run_name,output_filepath,route_info_allcar,darp_solution,origin_links,
    superzone_map,intrasuperzone_path_dic,average_value_of_time,external_factor)

In [ ]:
run_name='allhhcartrip1.5/'
output_filepath='Output/'+run_name
route_info_allcar15=pd.read_csv(output_filepath+'route_info.csv')
darp_solution_callcar15=pod.load_obj('darp_solutions',output_filepath)
average_value_of_time=round(traveler_trips.value_of_time.mean(),4)
external_factor='50'
dtd.write_darp_solution_to_file(run_name,output_filepath,route_info_allcar15,darp_solution_callcar15,origin_links,
    superzone_map,intrasuperzone_path_dic,average_value_of_time,external_factor)

In [ ]:
run_name='allhhcartrip1.25/'
output_filepath='Output/'+run_name
route_info_allcar125=pd.read_csv(output_filepath+'route_info.csv')
darp_solution_allcar125=pod.load_obj('darp_solutions',output_filepath)
average_value_of_time=round(traveler_trips.value_of_time.mean(),4)
external_factor='50'
dtd.write_darp_solution_to_file(run_name,output_filepath,route_info_allcar,darp_solution_allcar125,origin_links,
    superzone_map,intrasuperzone_path_dic,average_value_of_time,external_factor)

In [ ]:
run_name='modechoice1.1/'
output_filepath='Output/'+run_name
route_info_modechoice10=pd.read_csv(output_filepath+'route_info.csv')
darp_solution_modechoice10=pod.load_obj('darp_solutions',output_filepath)
average_value_of_time=round(traveler_trips.value_of_time.mean(),4)
output_filepath=output_filepath
external_factor='40'
dtd.write_darp_solution_to_file(run_name,output_filepath,route_info_modechoice10,darp_solution_modechoice10,origin_links,
    superzone_map,intrasuperzone_path_dic,average_value_of_time,external_factor)

# dtd.route_to_vehiclepathdat(route_info_modechoice10,origin_links,output_filepath,
#                             vehicle_filepath,path_filepath,superzone_map,
#                                 intrasuperzone_path_dic,external_vehicle_filepath,average_value_of_time)

In [ ]:
community_solution_allcar15=pod.load_obj('community_allcar15','Output/')

In [ ]:
community_solution_allcar15

In [ ]:
target_trips=traveler_trips.loc[(traveler_trips.tripmode<=6) &(traveler_trips.driver_passenger_flag>0)]
target_trips=target_trips[(target_trips.orig_taz!=target_trips.dest_taz) | (target_trips.trip_counter==1)]
target_trips=target_trips.append(traveler_trips[traveler_trips.tripmode>6])
target_trips=target_trips[(target_trips.orig_taz!=target_trips.dest_taz) | (target_trips.trip_counter==1)]
community_hh_list=[972044,972024,2212041,2212030,937968,2285653,1309779,1738127,2034377,2066988]
community_trips=pd.DataFrame()
for hh_id in community_hh_list:
#     print(len(target_trips[target_trips.hh_id==hh_id]))
    community_trips=community_trips.append(target_trips[target_trips.hh_id==hh_id])
sorted_trips=community_trips.sort_values('starttime')
pod.analysis_one_hh_result(937968,[community_solution_allcar15],community_trips,5,Vehicular_Skim_Dict,superzone_map)

In [ ]:
pod.plot_route_info_schedule(community_solution_allcar15['route_info'],community_trips,5)